In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
from ISLP.models import (ModelSpec as MS, summarize, poly)

# ------ please also import os, numpy,  pandas,  matplotlib.pyplot, statsmodels.api here:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
#------Set Working Directory (use os.chdir()):
os.chdir('/Users/aithadishanez/MGSC 401/Group Project/Final_proj')
#---- load data:
df1 = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2009-2010")
df2 = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2010-2011")

# Append them
df = pd.concat([df1, df2], ignore_index=True)

print(df.shape)
df.head()

ModuleNotFoundError: No module named 'ISLP'

In [ ]:
# Check for missing values
df.isna().sum()

In [ ]:
243007/1067371

In [ ]:
df['Customer ID'].value_counts()

In [ ]:
df.info()

In [ ]:
df["InvoiceDate"] = df["InvoiceDate"].dt.date
df["InvoiceDate"]

In [ ]:
df.info()

In [ ]:
df = df[df["Quantity"] > 0]
df

NameError: name 'df' is not defined

Total orders per month to show seasonality patterns

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(orders_per_month['year_month'].astype(str),
         orders_per_month['n_orders'],
         marker='o')
plt.xticks(rotation=45)
plt.xlabel('Year-Month')
plt.ylabel('Number of orders (invoices)')
plt.title('Total orders per month')
plt.tight_layout()
plt.show()

Total number of orders per customer

In [ ]:
cust_orders['n_invoices_capped'] = cust_orders['n_invoices'].clip(upper=30)

plt.figure(figsize=(8, 4))
plt.hist(
    cust_orders['n_invoices_capped'],
    bins=30,
    edgecolor='black',
    color='purple'     # make bars purple
)
plt.xlabel('Number of invoices per customer (capped at 30)')
plt.ylabel('Number of customers')
plt.title('How many times do customers order?')
plt.tight_layout()
plt.show()



How many customers order at least once?

In [ ]:
import matplotlib.pyplot as plt

plt.bar(['No reorder (1 invoice)', 'Reordered (>=2 invoices)'],
        [total_customers - n_reorder_customers, n_reorder_customers])
plt.ylabel('Number of customers')
plt.title('How many customers reorder at least once?')
plt.show()


For our feature engineering, we can see things like is_holiday,also season and country=> since many countries we can do season

### Key feature engineering

In [ ]:
df = df.dropna(subset=["Customer ID"])
df.info()

In [ ]:
# Check countires
df['Country'].value_counts()

In [ ]:
# Simplify country
df['Country_simple'] = np.where(df['Country'] == 'United Kingdom', 'UK',
                         np.where(df['Country'] == 'Germany', 'Germany',
                         np.where(df['Country'] == 'EIRE', 'Ireland',
                         np.where(df['Country'] == 'France', 'France',
                                  'Other'))))


In [ ]:
df.info()

In [ ]:
df['InvoiceDate'].value_counts()

In [ ]:
# Convert InvoiceDate to datetime (keep as datetime, not string)
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

# Create holiday_type column (use str for comparison inside function)
def get_holiday_type(date):
    date_str = date.strftime("%Y-%m-%d")  # convert datetime to string for comparison
    if date_str in [
        "2010-12-25", "2010-12-26",
        "2011-12-25", "2011-12-26",
        "2009-12-25", "2009-12-26"
    ]:
        return "winter_holiday"

    elif date_str in [
        "2010-02-14", "2011-02-14", "2009-02-14",
        "2010-03-14", "2011-03-27", "2009-03-18",
        "2010-06-20", "2011-06-19", "2009-06-17"
    ]:
        return "secondary_holiday"

    elif date_str in [
        "2010-01-01", "2011-01-01", "2009-01-01"
    ]:
        return "new_year"

    else:
        return "none"

df["holiday_type"] = df["InvoiceDate"].apply(get_holiday_type)
# extract month
df['month'] = df['InvoiceDate'].dt.month

In [ ]:
df.info()

In [ ]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"]).dt.floor("D")

In [ ]:
df = df.sort_values(["Customer ID", "StockCode", "InvoiceDate"])


In [ ]:
# Next purchase date per customer-SKU
df["next_purchase_date"] = df.groupby(["Customer ID", "StockCode"])["InvoiceDate"].shift(-1)

# Days until next purchase
df["days_to_reorder"] = (df["next_purchase_date"] - df["InvoiceDate"]).dt.days

# choose benchmark for y
median=df["days_to_reorder"].median()
mean = df["days_to_reorder"].mean()
print(median, mean)

In [ ]:
# Target variable: 1 if reorder ≤median days, else 0
df["y_reorder"] = (df["days_to_reorder"] <= 42).astype(int)

# Remove rows without future purchase (cannot compute target)
df_model = df[df["next_purchase_date"].notnull()].copy()


In [ ]:
df_model['days_to_reorder'].value_counts()

In [ ]:
df_model.head(20)

In [ ]:
# Remove returns
df_model = df_model[df_model["Quantity"] > 0].copy() # we dont take into account returns
df_model.head()

In [ ]:

# Shift to get the previous purchase date
df_model["prev_purchase_date"] = df_model.groupby(["Customer ID", "StockCode"])["InvoiceDate"].shift(1)

# Compute days since last purchase
df_model["days_since_last"] = (
    df_model["InvoiceDate"] - df_model["prev_purchase_date"]
).dt.days

# Replace NaN days with -1 (first purchase)
df_model["days_since_last"] = df_model["days_since_last"].fillna(-1)


In [ ]:
df_model.head()

In [ ]:
df_model["y_reorder"].value_counts()

In [ ]:
df_model.info()

In [ ]:
# drop useless columns
df_model = df_model.drop(columns = ['Invoice', 'StockCode', 'Description', 'InvoiceDate', 'Customer ID', 'Country', 'next_purchase_date', 'prev_purchase_date', 'days_to_reorder'])
df_model.info()

In [ ]:
df_model1 = df_model.copy()
df_model1 = df_model1.drop(columns = ['days_since_last', 'month'])

In [ ]:
df_model1.describe()

In [ ]:
# Standardize data
from sklearn.preprocessing import StandardScaler

numeric_cols = ["Quantity", "Price"]

scaler = StandardScaler()
df_model1[numeric_cols] = scaler.fit_transform(df_model1[numeric_cols])


In [ ]:
# Get dummies
df_model1 = pd.get_dummies(df_model1, columns=["Country_simple"], drop_first=True)

df_model1['holiday_type'] = df_model1['holiday_type'].astype('category')
# Replace 'none' with NaN (so it will be ignored by get_dummies)
df_model1['holiday_type'] = df_model1['holiday_type'].replace('none', pd.NA)
# Create dummies
df_model1 = pd.get_dummies(df_model1, columns=['holiday_type'], dummy_na=False)


In [ ]:
import seaborn as sns
# Correlation heatmap
correlation_matrix = df_model1.corr() # for numeric columns only

plt.figure(figsize=(8, 6))
heatmap = sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap', pad=20)
plt.show()

### Logistic Regression Model (Generic Coefficients)

Logit form:

$\log \frac{p}{1-p} = B_0 + B_1 \cdot \text{Quantity} + B_2 \cdot \text{Price} + B_3 \cdot \text{Country\_Germany} + B_4 \cdot \text{Country\_Ireland} + ... + B_7 \cdot \text{SecondaryHoliday}$


In [ ]:
# Fix dummies dtypes
# Convert all boolean columns to int
bool_cols = df_model1.select_dtypes(include='bool').columns
df_model1[bool_cols] = df_model1[bool_cols].astype(int)


In [ ]:
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

# Features and target
X = df_model1.drop('y_reorder', axis=1)
y = df_model1['y_reorder']

# Fit logistic regression on training data
glm = sm.GLM(y, X, family=sm.families.Binomial())
results = glm.fit()

# Summary
print(results.summary())


In [ ]:
import statsmodels.api as sm

# Fit GLM as before
X = sm.add_constant(X)  # or your full X if not split
glm = sm.GLM(y, X, family=sm.families.Binomial())
results = glm.fit()

# Use summary2 to get a DataFrame
results_summary = results.summary2().tables[1]  # Coefficients table

# Filter for significant coefficients (p <= 0.05)
results_sig = results_summary[results_summary['P>|z|'] <= 0.05]
results_sig


### Results Analysis

In [ ]:
# Convert coefficients to exp
results_sig['odds_ratio'] = np.exp(results_sig['Coef.'])

results_sig


In [ ]:
#========= get probability for customer ID
probs = results.predict(X)
probs


NameError: name 'results' is not defined

In [ ]:
#=========set a threshold to make prediction
threshold=0.5
labels = np.where(probs > threshold, 1, 0)
labels

In [ ]:
from ISLP import confusion_table
#==========generate confusion matrix
cm = confusion_table(labels, y)
cm

In [ ]:
#=======create data frame
compare=pd.DataFrame({'Actual Dec': df_model1['y_reorder'], 'Prob': probs, 'Prediced Dec': labels})
compare

In [ ]:
#=======accuracy, specificity, sensitivity

tn, fp, fn, tp = cm.to_numpy().ravel()   # unpack confusion matrix

accuracy    = (tp + tn) / (tp + tn + fp + fn)
sensitivity = tp / (tp + fn)   # true positive rate, recall
specificity = tn / (tn + fp)   # true negative rate

print(f"Accuracy:    {accuracy:.3f}")
print(f"Sensitivity: {sensitivity:.3f}")
print(f"Specificity: {specificity:.3f}")

In [ ]:
#========= ROC curve

from sklearn.metrics import RocCurveDisplay, roc_auc_score
import matplotlib.pyplot as plt

#----- get prob
probs = results.predict(X)  # fitted probabilities

#----- compute AUC
auc = roc_auc_score(y, probs)
print(f"AUC: {auc:.3f}")

#-----draw the graph
RocCurveDisplay.from_predictions(y, probs)


#-----add title
plt.title("ROC Curve: Dating Logistic Regression")
plt.show()

### Logistics regression model 2

In [ ]:
df_model2 = df_model.copy()
# Create dummy: 1 if days_since_last <= 42, else 0
df_model2['has_prev_purchase'] = (df_model2['days_since_last'] <= 42).astype(int)
df_model2 = df_model2.drop(columns = ['days_since_last'])

In [ ]:
df_model2.describe()

In [ ]:
# Standardize data
from sklearn.preprocessing import StandardScaler

numeric_cols = ["Quantity", "Price"]

scaler = StandardScaler()
df_model2[numeric_cols] = scaler.fit_transform(df_model2[numeric_cols])


In [ ]:
# Get dummies
df_model2 = pd.get_dummies(df_model2, columns=["Country_simple", 'month'], drop_first=True)
# First, replace 'none' with NaN so get_dummies can drop it
df_model2['holiday_type'] = df_model2['holiday_type'].replace('none', pd.NA)

# Create dummies for holiday_type, drop NaN (i.e., 'none')
df_model2 = pd.get_dummies(df_model2, columns=['holiday_type'], dummy_na=False)



In [ ]:
import seaborn as sns
# Correlation heatmap
correlation_matrix = df_model2.corr() # for numeric columns only

plt.figure(figsize=(8, 6))
heatmap = sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap', pad=20)
plt.show()

In [ ]:
# Fix dummies dtypes
# Convert all boolean columns to int
bool_cols = df_model2.select_dtypes(include='bool').columns
df_model2[bool_cols] = df_model2[bool_cols].astype(int)


### Logistic Regression Model 2

**Logit form:**

$$
\log \frac{p}{1-p} = B_0
+ B_1 \cdot \text{Quantity}
+ B_2 \cdot \text{Price}
+ B_3 \cdot \text{has\_prev\_purchase}
+ ...
+ B_{19} \cdot \text{holiday\_type\_secondary\_holiday}
$$

In [ ]:
import statsmodels.api as sm

# X = all features (numeric + encoded categorical)
X2 = df_model2.drop('y_reorder', axis=1)
y2 = df_model2['y_reorder']

# Add intercept column
X2 = sm.add_constant(X2)

# Logistic regression (GLM with Binomial family)
glm = sm.GLM(y2, X2, family=sm.families.Binomial())
results = glm.fit()
results_summary=summarize(results)

results_summary


In [ ]:
results_sig= results_summary[results_summary['P>|z|']<=0.05]
results_sig

In [ ]:
# Convert coefficients to exp
results_sig['odds_ratio'] = np.exp(results_sig['coef'])

results_sig


In [ ]:
#========= get probability for customer ID
probs = results.predict(X2)
probs

In [ ]:
#=========set a threshold to make prediction
threshold=0.5
labels = np.where(probs > threshold, 1, 0)
labels

In [ ]:
from ISLP import confusion_table
#==========generate confusion matrix
cm = confusion_table(labels, y)
cm

In [ ]:
#=======create data frame
compare=pd.DataFrame({'Actual Dec': df_model2['y_reorder'], 'Prob': probs, 'Prediced Dec': labels})
compare

In [ ]:
#=======accuracy, specificity, sensitivity

tn, fp, fn, tp = cm.to_numpy().ravel()   # unpack confusion matrix

accuracy    = (tp + tn) / (tp + tn + fp + fn)
sensitivity = tp / (tp + fn)   # true positive rate, recall
specificity = tn / (tn + fp)   # true negative rate

print(f"Accuracy:    {accuracy:.3f}")
print(f"Sensitivity: {sensitivity:.3f}")
print(f"Specificity: {specificity:.3f}")

In [ ]:
#========= ROC curve

from sklearn.metrics import RocCurveDisplay, roc_auc_score
import matplotlib.pyplot as plt

#----- get prob
probs = results.predict(X2)  # fitted probabilities

#----- compute AUC
auc = roc_auc_score(y2, probs)
print(f"AUC: {auc:.3f}")

#-----draw the graph
RocCurveDisplay.from_predictions(y2, probs)


#-----add title
plt.title("ROC Curve: Dating Logistic Regression")
plt.show()